In [117]:
%pip install selenium
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import getpass





def initialize_webdriver():
    # Initialize the WebDrive
    driver_path = 'chromedriver-win64/chromedriver.exe'

    # Set up the Chrome web driver
    driver = webdriver.Chrome(driver_path)

    return driver





def login(driver):

    # Get User credentials with safety
    user = input("Enter the username: ")
    passwd = getpass.getpass("Enter your password: ")

    # Navigate to the LinkedIn login page
    driver.get('https://www.linkedin.com/login')

    # Wait for the page to load
    time.sleep(2)

    # Fill in the login form
    username = driver.find_element_by_id('username')
    username.send_keys(user)
    password = driver.find_element_by_id('password')
    password.send_keys(passwd)
    password.submit()

    # Wait for the home page to load
    time.sleep(2)






def search_people(driver):

    # Person's first and last name
    first_name = input("Enter the user's first name: ")
    last_name = input("Enter the user's last name: ")
    search_query = first_name+' '+last_name

    #Search on LinkedIn
    driver.get(f"https://www.linkedin.com/search/results/people/?keywords={search_query}")
    time.sleep(7)
    # Extract data from the first 10 search results
    profile_urls = []
    results = driver.find_elements_by_class_name('search-results-container')
    
    profiles = results[0].find_elements_by_class_name('reusable-search__result-container')[:10]
    for profile in profiles:
        profile_urls.append(profile.find_element_by_css_selector('a.app-aware-link').get_attribute('href'))

    return profile_urls





def get_data(driver,profile_urls):
    
    # Initialize a table with given columns
    top_profiles_data = pd.DataFrame(columns=['Name','Location','Summary','About','Activity','Experience','Education','Skills',
                                              'Interests','Recommendations','Projects','Licenses & certifications'])
    

    # Work with 1 profile at a time
    for url_num in range(len(profile_urls)):
        datas={}    

        # Get Profile page
        driver.get(profile_urls[url_num])

        #Wait for page to load
        time.sleep(5)

        # Get the page source for HTML parser
        driver_html = driver.page_source

        # Get the html page using Beautiful Soup
        soup = BeautifulSoup(driver_html, 'html.parser')


        # Retrive basic information like Name, Summary, Location
        primary_information = soup.find('div',{'class':'mt2 relative'}).text

        # Remove redundant characters like space and "\n"
        primary_text = re.sub(r'\n','',primary_information).split('    ')
        primary_text = [''.join(re.sub(r'  ','',text)).strip() for text in primary_text if text]

        primary_text.remove('Contact info') #Redundant info

        # Removing not required data like type of connections etx
        for text in primary_text:
            if text[0].isdigit():
                primary_text.remove(text)

        #Intializing our data with basic information gathered
        datas = {'Name':primary_text[0],
                'Location':primary_text[len(primary_text)-1],
                'Summary':', '.join(primary_text[1:(len(primary_text)-1)]),}
        
        # Getting remaining information by section-wise
        secondary_information = soup.find_all('section')
        sections = ['About','Activity','Experience','Education','Licenses & certifications','Skills','Interests','Recommendations','Projects']

        # Iterating through all sections
        for section in secondary_information:
            span_texts = section.find_all('span',{"class":"visually-hidden"})
            secondary_text = []

            # Iterating through all span classes containing text
            for texts in span_texts:
                if texts.text not in secondary_text:

                    # Removing trailing and lagging spaces
                    information= re.sub(r'\n',' ',texts.text).strip()
                    secondary_text.append(information)
            
            # Combining all data found under 1 section to make our dataframe valid with same length arrays
            secondary_text = [''.join(re.sub(r'  ',' ',info)).strip() for info in secondary_text if info]

            # Check for data found along with sections
            if len(secondary_text)>=2:
                if secondary_text[0] in sections:

                    # Combining data for our dataframe
                    datas[secondary_text[0]] = ', '.join(secondary_text[1:])

        # Updating out initialized dataframe "top_profiles_data"
        top_profiles_data = pd.concat([top_profiles_data,pd.DataFrame(datas,index=[0])],axis=0)

    top_profiles_data.index = profile_urls
    return top_profiles_data

        


def main():

    driver = initialize_webdriver()

    login(driver)

    profile_urls = search_people(driver)

    top_profiles_data = get_data(driver,profile_urls)

    print('Data Stored!')

    driver.close()

    top_profiles_data.to_csv('top_profile_data')
    
    return top_profiles_data

top_profiles_data = main()

Data Stored!


In [2]:
# Just for Refrence
top_profiles_data

,Name,Location,Summary,About,Activity,Experience,Education,Skills,Interests,Recommendations,Projects,Licenses & certifications
https://www.linkedin.com/in/daniel-richards-59b6801a4?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC_ME0cBNLD-h0kEHgNz4YzVvW_mU-fQaYs,Daniel Richards,United Kingdom,"Senior Software Developer, SPG Software, Newca...","Top skills, TypeScript • HTML • Cascading Styl...","224 followers, 3mo, 5mo","Senior Software Developer, SPG Software · Full...","Newcastle College, Music Enterprise, 2015 - 20...","Express.js, 3 experiences across SPG Software ...","Top Voices, Companies, Groups, Schools, Bill G...","Received, Given, Nothing to see for now, Recom...",NaN,"Software Development From A to Z, Udemy, Issue..."
https://www.linkedin.com/in/daniel-richards-99049314?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAALrIrwBH6CXg8xJT-3ptQ71KuX31qj7q6A,Daniel (Danny) Richards,Greater Buenos Aires,"Chairman of the Board - LatinRisk Argentina, T...","LatinRisk Argentina - t-Risk platform, is a ne...","6,731 followers, 3d, 4d, 5d",VP de Cuentas Corporativas South America (exce...,Fundacion de Altos Estudios en Ciencias Comerc...,"Gestión operativa, 4 experiences across t-Risk...","Top Voices, Companies, Groups, Newsletters, Sc...","Received, Given, Tácito Augusto Silva Leite, T...",Digitalización (bajo Standard ISO 31000)- del ...,Analista Senior de Gestion y Tratamiento de Ri...
https://www.linkedin.com/in/talldan?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAALhU1EBGj0Lm8BPg17_H4kWqEWbOCX9At8,Daniel Richards,Greater Perth Area,"JavaScript Engineer at Automattic, Automattic,...",Software developer specialising in JavaScript ...,281 followers,"JavaScript Engineer, Automattic, Jun 2018 - Pr...","University of Westminster, BSc Hon, Computer G...","CSS, Endorsed by William Thirkettle who is hig...","Companies, Groups, Schools, Microsoft, 21,962,...","Received, Given, Nothing to see for now, Recom...","Lyroke, Apr 2014 - Present, Associated with We...",NaN
https://www.linkedin.com/in/dan-rds?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABGe8LcBySgbMtd3-L2dIodUjUK0CVd9l7Q,Daniel Richards,"Berlin, Berlin, Germany","Software Engineer at Google, Google, Universit...",Software engineering candidate with education ...,129 followers,"Software Engineer Resident, Google · Full-time...","University of California, Santa Cruz, Bachelor...","Software Development, 2 endorsements, Python (...","Top Voices, Companies, Groups, Schools, Jim Cr...",NaN,NaN,NaN
https://www.linkedin.com/in/richardsdaniel?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAA4ZdHkBF7hebCdBdHx5jN2ANay2wJMjPDs,Daniel Richards,"London, England, United Kingdom","3rd, Director of IT and Digital Recruitment, h...",I am a Public and Not for Profit focused recru...,"5,652 followers, 5d, 1w, hashtag","Morgan Law, 8 yrs 8 mos, Director - IT and Dig...","University of Exeter, Bachelor's Degree, Histo...","Leadership, 33 endorsements, Political Campaig...","Companies, Groups, Schools, BBC, 2,127,873 fol...","Received, Given, Deborah Hale, Third degree co...","Exeter For Women's Fortnight, Mar 2014 - Sep 2...",NaN
https://www.linkedin.com/in/daniel-richards-3a94a326?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAVpUHEBXeI653-ieO5RBzmxNz8fbUGfswI,Daniel Richards,"New York, New York, United States","(He/Him), 3rd, Director @ Persefoni | Carbon A...",I am deeply grateful and energized to integrat...,"3,638 followers, 1w, hashtag, 3w","Director, Large Enterprise Accounts, Persefoni...","Roberts Wesleyan University, Bachelor's degree...","Sales, Endorsed by John Wasserman and 6 others...","Top Voices, Companies, Groups, Newsletters, Al...","Received, Given, John Amici, SVP of Sales at G...",NaN,"Ambassador, Donek Snowboards Inc, Issued Sep 2..."
https://www.linkedin.com/in/daniel-richards-9a789327?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAWlws8BeJl_JonKZirRNYcH-7BnlIhNuwU,Daniel Richards,"Dubai, United Arab Emirates","3rd, MENA Economist at Emirates NBD, Emirates ...",C